In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
import os

from sklearn.metrics import mean_squared_error 

import pandas as pd
import numpy as np
import re
import neptune

from dotenv import load_dotenv

load_dotenv()

True

# Example

In [3]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo-1106", openai_api_key=os.getenv('OPENAI_API_KEY'))
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "bread"})

'Why was the bread always so calm? \nBecause it kneaded to relax!'

# Parameters

In [4]:
post_type = 'post_travel'
narcism_type = 'adm'
model_used = "gpt-3.5-turbo-1106"
iterations = 10
number_of_shots = 5 # somewhere between 3 and 10
model_role = "You are a psychologist and you are assessing a patient's Narcissism. The patient is talking about their recent travel. Return only float number between 1 and 6."
train_path = "../data/split/train.csv"
validate_path = "../data/split/validate.csv"

# Code

Here we used the most basic implementation, there is also option to use Dynamic few-shot prompting, but to my knowledge is not needed is this context as we have only one type of posts.

In [5]:
# Get split data using pandas
df = pd.read_csv(train_path)

# Get the dictionary of the first x posts
example = df[[post_type,narcism_type]].iloc[0:number_of_shots]

example = example.to_dict(orient='records')

# Change the value name
for i in range(len(example)):
    example[i]['post'] = example[i].pop(post_type)
    example[i]['narcissism'] = example[i].pop(narcism_type)

example

[{'post': 'I wish I could travel 24/7 and get paid for it',
  'narcissism': 1.444},
 {'post': "Vacations are pricey these days but so worth it! I had the most amazing weekend at ABC resort. Everything about this place screams relaxation and luxury. I'm definitely going back next year. Would you like to come with me?",
  'narcissism': 3.889},
 {'post': 'I recently visited beautiful Stratford upon Avon as a pit-stop on my way to Minehead, Somerset. I made a point to leave my immediate surroundings and find the birthplace of Shakespeare. I found it interesting but ultimately over-commercialised.',
  'narcissism': 3.444},
 {'post': "I have just visited Marrakesh.The scenery is like being in Mars.The soil is do red and there's not a person around for miles. Then you will come across a shepherd all alone with his flock. It makes you wonder how he gets food.The transport there is mainly donkey and cart.",
  'narcissism': 3.667},
 {'post': "I travel a lot for work, and I get to see all sorts o

In [6]:
model = ChatOpenAI(model=model_used, openai_api_key=os.getenv('OPENAI_API_KEY'))


In [7]:
# This is a prompt template used to format each example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{post}"),
        ("ai", "result: {narcissism}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=example,
)

# print(few_shot_prompt.format())

#### Use train and validate dataset!!!

In [8]:
test = df[[post_type,narcism_type]].iloc[4]
input = test.iloc[0]
input

"I travel a lot for work, and I get to see all sorts of cool places. I didn't think the American south had as many hidden gems as it did, I think it's a beautiful region with some crazy cool people."

In [9]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Return a narcissism number between 1 and 6."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

final_prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Return a narcissism number between 1 and 6.')), FewShotChatMessagePromptTemplate(examples=[{'post': 'I wish I could travel 24/7 and get paid for it', 'narcissism': 1.444}, {'post': "Vacations are pricey these days but so worth it! I had the most amazing weekend at ABC resort. Everything about this place screams relaxation and luxury. I'm definitely going back next year. Would you like to come with me?", 'narcissism': 3.889}, {'post': 'I recently visited beautiful Stratford upon Avon as a pit-stop on my way to Minehead, Somerset. I made a point to leave my immediate surroundings and find the birthplace of Shakespeare. I found it interesting but ultimately over-commercialised.', 'narcissism': 3.444}, {'post': "I have just visited Marrakesh.The scenery is like being in Mars.The soil is do red and there's not a person around for miles. Then you will come a

In [10]:
chain = final_prompt | model

ai_message = chain.invoke({"input": input})

ai_message


AIMessage(content='result: 1.556', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 356, 'total_tokens': 362}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_89448ee5dc', 'finish_reason': 'stop', 'logprobs': None})

# Analyze the results

In [11]:
r = ai_message.content
match = re.search(r'\d+\.\d+', r)
if match:
    response = float(match.group())
else:
    response = None
response

1.556

In [12]:
test.iloc[1]

1.222

In [13]:
y_pred = []
y_true = []
y_pred.append(response)
y_true.append(test.iloc[1])

In [14]:
mse = mean_squared_error(y_true=y_true, y_pred=y_pred)
mse

0.11155600000000004

# Implementation

In [15]:
# functions

# get random x posts
def get_random_x_posts(path, post_type, narcism_type, number_of_posts):

    df = pd.read_csv(path)
    example = df[[post_type,narcism_type]].sample(number_of_posts)
    example = example.to_dict(orient='records')

    # Change the value name
    for i in range(len(example)):
        example[i]['post'] = example[i].pop(post_type)
        example[i]['narcissism'] = example[i].pop(narcism_type)

    return example

# create a few shot prompt
def create_few_shot_prompt(example):
    example_prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "{post}"),
            ("ai", "narcissism: {narcissism}"),
        ]
    )
    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=example,
    )
    return few_shot_prompt

# create a final prompt
def create_final_prompt(few_shot_prompt,model_role):
    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", model_role),
            few_shot_prompt,
            ("human", "{input}"),
        ]
    )
    return final_prompt

# get float number from a string
def get_float(text):
    # Use regular expression to find numerical value
    match = re.search(r'\d+\.\d+', text)
    if match:
        float_number = float(match.group())
        if float_number is not None:
            return float_number
        else:
            print(f"Wrong input: {text}")
    else:
        return None

# get the response
def get_response(final_prompt, model, input):
    chain = final_prompt | model
    ai_message = chain.invoke({"input": input})
    response = ai_message.content
    print(response)
    # get a float number from a string
    response = get_float(response)
    return response

# get the mean squared error
def get_mse(y_pred, y_true):
    mse = mean_squared_error(y_true=y_true, y_pred=y_pred)
    return mse

Add Neptune experiment observation

In [16]:
# Run the functions
run = neptune.init_run(project = os.getenv('NEPTUNE_PROJECT'),
                       api_token = os.getenv('NEPTUNE_API_TOKEN'),
                       source_files=["few_shot_test.ipynb"],
                       tags=["few-shot", "narcissism", narcism_type])
# TODO: Add logging of input posts (and responses?) to Neptune
run["type"] = "Few-shot learning"
params = {
    "model": model_used,
    "narc_type": narcism_type,
    "post_type": post_type,
    "prompt": model_role,
    "shots": number_of_shots
}
run["model/parameters"] = params
y_pred = []
y_true = []
for i in range(iterations):
    example = get_random_x_posts(train_path, post_type, narcism_type, number_of_shots)
    few_shot_prompt = create_few_shot_prompt(example)
    input_dic = get_random_x_posts(validate_path, post_type, narcism_type, 1)
    input = input_dic.pop(0)
    final_prompt = create_final_prompt(few_shot_prompt,model_role)
    response = get_response(final_prompt, model, input.get('post'))
    y_pred.append(response)
    y_true.append(input.get('narcissism'))

mse = get_mse(y_pred, y_true)
print(mse)
run["mse"] = mse
run.stop()

[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/NarcisissticTwitter/Twitter/e/TWIT-29
narcissism: 3.556
narcissism: 2.111
narcissism: 3.111
narcissism: 5.0
narcissism: 1.444
narcissism: 1.111
narcissism: 3.111
narcissism: 3.667
narcissism: 1.111
narcissism: 3.222
2.1385507999999995
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 1 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/NarcisissticTwitter/Twitter/e/TWIT-29/metadata
